In [6]:
#inifinte pendulum
import sympy as sp
from sympy import cos, sin
from sympy.physics.mechanics import dynamicsymbols
import dill
dill.settings['recurse'] = True


ms = [.1, .1, .1, .1, .1]
Ls = [.5, .5, .5, .5, .5]
g = 9.81

thetas = [dynamicsymbols(f"theta{i}") for i in range(len(Ls))]
thetaDots = [dynamicsymbols(f"theta{i}", 1) for i in range(len(Ls))]
thetaDDots = [dynamicsymbols(f"theta{i}", 2) for i in range(len(Ls))]
for i, L in enumerate(Ls):
    theta = thetas[i]

    if i == 0:
        x = L*sin(theta)
        y = -L*cos(theta)
        pos = [x,y]
    else:
        pos.append(x+L*sin(theta))
        pos.append(y-L*cos(theta))
vel = [sp.diff(pos[i], "t") for i in range(len(pos))]

j=0
for i in range(0, len(vel)-1, 2):
    if i == 0:
        KE = ms[j]*(vel[i]**2+vel[i+1]**2)/2
        PE = ms[j]*g*pos[i+1]
    else:
        KE = KE + ms[j]*(vel[i]**2+vel[i+1]**2)/2
        PE = PE+ms[j]*g*pos[i+1]
    j+=1
L = KE-PE
LEs = [sp.diff(sp.diff(L,thetaDots[i]), 't')-sp.diff(L,thetas[i]) for i in range(len(Ls))]
print('solving')
sol = sp.solve(LEs, thetaDDots)
print('simplifying')
thetaDDots_F = [sp.lambdify(thetas+thetaDots, sol[thetaDDots[i]].simplify()) for i in range(len(Ls))]
for i in range(len(Ls)):
    dill.dump(thetaDDots_F[i], open(f"thetaddot{i}_F", "wb"))

solving
simplifying


In [108]:
#run infinite pendulums
import matplotlib.pyplot as plt
import dill
import numpy as np

m = 1
Ls = [1, 1, 1]
g = 9.81
thetaddots_F = [dill.load(open(f"thetaddot{i}_F", "rb")) for i in range(len(Ls))]

class animation():
    def __init__(self):
            self.fig, self.ax = plt.subplots()
            self.ax.set_xlim(-3,3)
            self.ax.set_ylim(-3,3)
            self.init = False
            self.lines = [i for i in range(len(Ls))]
            self.trace = 1
            self.ax.set_title("T = ")
            self.x_history = []
            self.y_history = []

    def animate(self, state, T):
        thetas = [state[i][0] for i in range(len(Ls))]
        for i, L in enumerate(Ls):
            theta = thetas[i]
            if i == 0:
                x = L*sin(theta)
                y = -L*cos(theta)
                pos = [x,y]
            else:
                pos.append(x+L*sin(theta))
                pos.append(y-L*cos(theta))
        self.x_history.append(pos[-1])
        self.y_history.append(pos[-2])
        if len(self.x_history) > 50:
            self.x_history.pop(0)
            self.y_history.pop(0)
        if not self.init:
            for i in range(0, len(Ls)-1, 2):
                if i == 0: self.lines[i], = self.ax.plot([0,pos[i]], [0,pos[i+1]])
                else: self.lines[i], = self.ax.plot([pos[i-2],pos[i]], [pos[i-1],pos[i+1]])
            self.trace, = self.ax.plot(self.x_history, self.y_history)
            self.init = True
        else:
            for i in range(0, len(Ls)-1, 2):
                if i == 0: self.lines[i].set_data([0,pos[i]], [0,pos[i+1]])
                else: self.lines[i].set_data([pos[i-2],pos[i]], [pos[i-1],pos[i+1]])
            self.trace.set_data(self.x_history, self.y_history)

def f(state, force):
    thetas = [state[i][0] for i in range(len(Ls))]
    thetaDots = [state[i][0] for i in range(len(Ls), len(Ls)*2)]
    ins = thetas+thetaDots
    thetaDDots = [thetaddots_F[i](*ins) for i in range(len(Ls))]
    xdot = np.array(thetaDots+thetaDDots)[:, np.newaxis]
    return xdot


def rk4_step(state, force, Ts):
    # Integrate ODE using Runge-Kutta RK4 algorithm
    F1 = f(state, force)
    
    F2 = f(state + Ts / 2 * F1, force)
    F3 = f(state + Ts / 2 * F2, force)
    F4 = f(state + Ts * F3, force)
    state += Ts / 6 * (F1 + 2 * F2 + 2 * F3 + F4)
    return state

def update():
    #ani = animation()
    Ts = .05
    T = 0
    state = np.array([[90*3.14/180], [-130*3.14/180], [60*3.14/180],[0],[0],[0]]).astype("float64")
    for i in range(int(30/Ts)):
        T = T+Ts
        state = rk4_step(state,0,Ts)
        print(state[0,0])
        #ani.animate(state, T)
update()

1.55769010322331
1.5202471975261354
1.456077982112799
1.3624447348080528
1.235909738098963
1.0749103621944325
0.8855889686188096
0.6830979308604267
0.4827817836810544
0.29588127100524864
0.1321642915872048
0.0002931692105349326
-0.09505979867818165
-0.15677805894250746
-0.19843880270760988
-0.24288106496287096
-0.31225047024030955
-0.4228349508862748
-0.5923016641015126
-0.820822225990193
-1.0097822837388986
-1.1099897062834325
-1.1385731821291443
-1.1080656703924499
-1.0320651967479626
-0.9260658999179072
-0.8011809219050265
-0.6618943964320603
-0.5084936939523013
-0.3402320600514116
-0.15824084773055547
0.03240098286439813
0.22323312330985706
0.4060125158767259
0.5762575852351783
0.733922908001178
0.8818839701662144
1.023929009386247
1.161985938500025
1.289953208534813
1.3877644459464709
1.4347235018670657
1.425024133326718
1.3600166571920593
1.237390900153141
1.041882014994804
0.7460708228112768
0.4529428602042665
0.2349591517048963
0.05957264132960316
-0.08809455742591032
-0.213212

In [6]:
#triple pendulum
import matplotlib.pyplot as plt
import sympy as sp
from math import cos, sin, pi
from sympy import cos, sin
from sympy.physics.mechanics import dynamicsymbols
import numpy as np

L1, L2, L3, m1, m2, m3, g = sp.symbols("L1, L2, L3, m1, m2, m3, g")
L1, L2, L3, m1, m2, m3, g, mu = 1.2, 1, .5, 1, 1, 1, 9.81, 0.1
theta1, theta2, theta3 = dynamicsymbols("theta1, theta2, theta3")
thetadot1, thetadot2, thetadot3 = dynamicsymbols("theta1, theta2, theta3", 1)
thetaddot1, thetaddot2, thetaddot3 = dynamicsymbols("theta1, theta2, theta3", 2)


x1 = L1*sin(theta1)
y1 = -L1*cos(theta1)
x2 = x1+L2*sin(theta2)
y2 = y1-L2*cos(theta2)
x3 = x2+L3*sin(theta3)
y3 = y2-L3*cos(theta3)
pos = [x1,y1,x2,y2,x3,y3]

vel = [sp.diff(pos[i], "t") for i in range(len(pos))]
xdot1, ydot1, xdot2, ydot2, xdot3, ydot3 = vel

KE = m1*(xdot1**2+ydot1**2)/2 + m2*(xdot2**2+ydot2**2)/2 + m3*(xdot3**2+ydot3**2)/2
PE = m1*g*y1+m2*g*y2+m3*g*y3
L = KE-PE
LE1 = sp.diff(sp.diff(L,thetadot1), 't')-sp.diff(L,theta1) #- mu*thetadot1
LE2 = sp.diff(sp.diff(L,thetadot2), 't')-sp.diff(L,theta2)# - 6*np.pi*mu*.1*thetadot2
LE3 = sp.diff(sp.diff(L,thetadot3), 't')-sp.diff(L,theta3)
sol = sp.solve((LE1,LE2,LE3), (thetaddot1, thetaddot2, thetaddot3), dict=False)
var = [theta1, theta2, theta3, thetadot1, thetadot2, thetadot3]
thetaddot1_F, thetaddot2_F, thetaddot3_F = [sp.lambdify(var, sol[thetaddot1].simplify()), sp.lambdify(var, sol[thetaddot2].simplify()), sp.lambdify(var, sol[thetaddot3].simplify())]

import dill
dill.settings['recurse'] = True
dill.dump(thetaddot1_F, open("thetaddot1_F", "wb"))
dill.dump(thetaddot2_F, open("thetaddot2_F", "wb"))
dill.dump(thetaddot3_F, open("thetaddot3_F", "wb"))

In [60]:
#Double pendulum

import matplotlib.pyplot as plt
import sympy as sp
from math import cos, sin, pi
from sympy import cos, sin
from sympy.physics.mechanics import dynamicsymbols
import numpy as np

L1, L2, m1, m2, g = sp.symbols("L1, L2, m1, m2, g")
L1, L2, m1, m2, g, mu = 1, 1, 1, 1, 9.81, 0.1
theta1, theta2 = dynamicsymbols("theta1, theta2")
thetadot1, thetadot2 = dynamicsymbols("theta1, theta2", 1)
thetaddot1, thetaddot2 = dynamicsymbols("theta1, theta2", 2)


x1 = L1*sin(theta1)
y1 = -L1*cos(theta1)
x2 = x1+L2*sin(theta2)
y2 = y1-L2*cos(theta2)
pos = [x1,y1,x2,y2]

vel = [sp.diff(pos[i], "t") for i in range(len(pos))]
xdot1, ydot1, xdot2, ydot2 = vel

KE = m1*(xdot1**2+ydot1**2)/2+m2*(xdot2**2+ydot2**2)/2
PE = m1*g*y1+m2*g*y2
L = KE-PE
LE1 = sp.diff(sp.diff(L,thetadot1), 't')-sp.diff(L,theta1) #- mu*thetadot1
LE2 = sp.diff(sp.diff(L,thetadot2), 't')-sp.diff(L,theta2)# - 6*np.pi*mu*.1*thetadot2
sol = sp.solve((LE1,LE2), (thetaddot1, thetaddot2), dict=False)
thetaddot1_F, thetaddot2_F = [sp.lambdify([theta1, theta2, thetadot1,thetadot2], sol[thetaddot1].simplify()), sp.lambdify((theta1, theta2, thetadot1,thetadot2), sol[thetaddot2].simplify())]

import dill
dill.settings['recurse'] = True
dill.dump(thetaddot1_F, open("thetaddot1_F", "wb"))
dill.dump(thetaddot2_F, open("thetaddot2_F", "wb"))

In [2]:
class animation():
    def __init__(self):
            self.fig, self.ax = plt.subplots()
            self.ax.set_xlim(-3,3)
            self.ax.set_ylim(-3,3)
            self.init = False
            self.line1 = 1
            self.line2 = 1
            self.line3 = 1
    def animate(self, state):
        theta1 = state[0][0]
        theta2 = state[1][0]
        theta3 = state[2][0]

        x1 = L1*np.sin(theta1)
        y1 = L1*np.cos(theta1)
        x2 = x1+L2*np.sin(theta2)
        y2 = y1+L2*np.cos(theta2)
        x3 = x2+L3*np.sin(theta3)
        y3 = y2+L3*np.cos(theta3)

        if not self.init:
            self.ax.set_xlim(-3,3)
            self.ax.set_ylim(-3,3)
            self.line1, = self.ax.plot((0,x1), (0,y1))
            self.line2, = self.ax.plot((x1,x2),(y1,y2))
            self.line3, = self.ax.plot((x2,x3),(y2,y3))
            self.init = True
        else:
            self.line1.set_data((0,x1), (0,y1))
            self.line2.set_data((x1,x2),(y1,y2))
            self.line3.set_data((x2,x3),(y2,y3))
            plt.pause(0.01)

In [111]:
def f(state, force):
    theta1 = state[0][0]
    theta2 = state[1][0]
    theta3 = state[2][0]
    thetadot1 = state[3][0]
    thetadot2 = state[4][0]
    thetadot3 = state[5][0]
    thetaddot1 = thetaddot1_F(theta1, theta2, theta3, thetadot1, thetadot2, thetadot3)
    thetaddot2 = thetaddot2_F(theta1, theta2, theta3, thetadot1, thetadot2, thetadot3)
    thetaddot3 = thetaddot3_F(theta1, theta2, theta3, thetadot1, thetadot2, thetadot3)
    xdot = np.array([[thetadot1], [thetadot2], [thetadot3], [thetaddot1], [thetaddot2], [thetaddot3]])
    #print(xdot)
    return xdot

def rk4_step(state, force, Ts):
    # Integrate ODE using Runge-Kutta RK4 algorithm
    F1 = f(state, force)
    F2 = f(state + Ts / 2 * F1, force)
    F3 = f(state + Ts / 2 * F2, force)
    F4 = f(state + Ts * F3, force)
    print(state)
    print(F1)
    state += Ts / 6 * (F1 + 2 * F2 + 2 * F3 + F4)
    return state

def update():
    #ani = animation()
    Ts = .1
    state = np.array([[181*3.14/180],[0], [0],[0],[0],[0]]).astype("float64")
    for i in range(10*20):
        state = rk4_step(state,0,Ts)
        #ani.animate(state)
update()

[[3.15744444]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.46626436]
 [ 0.46620578]
 [-0.        ]]
[[3.15979482e+00]
 [2.31195037e-03]
 [3.80828182e-05]
 [4.73883191e-02]
 [4.58573188e-02]
 [1.52326302e-03]]
[[0.04738832]
 [0.04585732]
 [0.00152326]
 [0.49005182]
 [0.44498569]
 [0.04457684]]
[[3.16708993e+00]
 [9.04653636e-03]
 [5.90804733e-04]
 [9.99025836e-02]
 [8.81238989e-02]
 [1.15295391e-02]]
[[0.09990258]
 [0.0881239 ]
 [0.01152954]
 [0.57243348]
 [0.40060628]
 [0.16568529]]
[[3.18018081e+00]
 [1.98050955e-02]
 [2.84098197e-03]
 [1.64851133e-01]
 [1.26865488e-01]
 [3.61936612e-02]]
[[0.16485113]
 [0.12686549]
 [0.03619366]
 [0.74586224]
 [0.38504255]
 [0.33183036]]
[[3.20084585]
 [0.03449415]
 [0.00841701]
 [0.2538107 ]
 [0.16841825]
 [0.0782059 ]]
[[0.2538107 ]
 [0.16841825]
 [0.0782059 ]
 [1.06310409]
 [0.46895278]
 [0.50783358]]
[[3.23231732]
 [0.0540152 ]
 [0.01905832]
 [0.3846061 ]
 [0.22634501]
 [

In [4]:
x = [0,1,2,3]
for i in range(1, 3+1, 1): 
    print(x[i])

1
2
3
